In [75]:
from pyciemss import PetriNetODE


In [76]:
import os

from pyciemss.PetriNetODE.base import BetaNoisePetriNetODESystem
from pyciemss.PetriNetODE.events import Event, StartEvent, LoggingEvent, ObservationEvent, StaticParameterInterventionEvent
import pyciemss

Let's see what directory we're in

In [78]:
os.getcwd()

'/home/chads/git/pyciemss/notebook'

Most of our files are relative to `test/models/`, so we need to go up a directory.

In [79]:
PYCIEMSS_PATH = ".."
STARTERKIT_PATH = os.path.join(PYCIEMSS_PATH, "test/models/starter_kit_examples/")
MIRA_PATH = os.path.join(PYCIEMSS_PATH, "test/models/evaluation_examples/")

filename = "CHIME-SIR/model_petri.json"
filename = os.path.join(STARTERKIT_PATH, filename)

model = BetaNoisePetriNetODESystem.from_mira(filename)

start_event = StartEvent(0.0, {"S": 0.99, "I": 0.01, "R": 0.0})
model.load_event(start_event)

tspan = range(1, 10)
logging_events = [LoggingEvent(t) for t in tspan]
model.load_events(logging_events)

solution = model()

# See that the solution returns a dictionary where each value has length 10, one for each logging event.
assert len(solution["I"]) == len(solution["R"]) == len(solution["S"]) == len(tspan)

solution

{'I': tensor([0.0096, 0.0091, 0.0085, 0.0080, 0.0074, 0.0069, 0.0063, 0.0058, 0.0053]),
 'R': tensor([0.0092, 0.0180, 0.0263, 0.0341, 0.0414, 0.0482, 0.0544, 0.0601, 0.0653]),
 'S': tensor([0.9812, 0.9729, 0.9651, 0.9579, 0.9511, 0.9450, 0.9393, 0.9341, 0.9295])}

In [4]:
# Remove logging events
model.remove_logging_events()

# Add observations
observation_events = [ObservationEvent(1.1, {"S": 0.9, "I": 0.09, "R": 0.01}), 
                      ObservationEvent(2.1, {"S": 0.8, "I": 0.18, "R": 0.02}),
                      ObservationEvent(3.1, {"S": 0.7, "I": 0.27, "R": 0.03}),
                      ObservationEvent(4.1, {"S": 0.6, "I": 0.36, "R": 0.04})]

model.load_events(observation_events)

In [5]:
model._static_events

[StartEvent(time=0.0, initial_state={'S': tensor(0.9900), 'I': tensor(0.0100), 'R': tensor(0.)}),
 ObservationEvent(time=1.100000023841858, observation={'S': tensor(0.9000), 'I': tensor(0.0900), 'R': tensor(0.0100)}),
 ObservationEvent(time=2.0999999046325684, observation={'S': tensor(0.8000), 'I': tensor(0.1800), 'R': tensor(0.0200)}),
 ObservationEvent(time=3.0999999046325684, observation={'S': tensor(0.7000), 'I': tensor(0.2700), 'R': tensor(0.0300)}),
 ObservationEvent(time=4.099999904632568, observation={'S': tensor(0.6000), 'I': tensor(0.3600), 'R': tensor(0.0400)})]

In [6]:
# Show that inference works.

from pyro.infer.autoguide import AutoNormal
from pyro.poutine import block
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import pyro

guide = AutoNormal(model)
optim = Adam({'lr': 0.03})
loss_f = Trace_ELBO(num_particles=1)
verbose = True

svi = SVI(model, guide, optim, loss=loss_f)

pyro.clear_param_store()

for j in range(100):
    # calculate the loss and take a gradient step
    # Passing a data argument to svi.step() will condition the model on the data.
    loss = svi.step()
    if verbose:
        if j % 25 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss))

[iteration 0001] loss: 22.7344
[iteration 0026] loss: 23.4423
[iteration 0051] loss: 20.6464
[iteration 0076] loss: 20.7368


In [7]:
# Remove the observation events
model.remove_observation_events()

In [8]:
# Load some static parameter intervention events
model.load_event(StaticParameterInterventionEvent(2.99, "beta", 0.0))
model.load_event(StaticParameterInterventionEvent(4.11, "beta", 10.))

# Load the logging events again
model.load_events(logging_events)

In [9]:
model._static_events

[StartEvent(time=0.0, initial_state={'S': tensor(0.9900), 'I': tensor(0.0100), 'R': tensor(0.)}),
 LoggingEvent(time=1),
 LoggingEvent(time=2),
 StaticParameterInterventionEvent(time=2.990000009536743, parameter=beta, value=0.0),
 LoggingEvent(time=3),
 LoggingEvent(time=4),
 StaticParameterInterventionEvent(time=4.110000133514404, parameter=beta, value=10.0),
 LoggingEvent(time=5),
 LoggingEvent(time=6),
 LoggingEvent(time=7),
 LoggingEvent(time=8),
 LoggingEvent(time=9)]

In [10]:
model._observation_var_names

[]

In [11]:
# use pyro predictive
from pyro.infer import Predictive

# Get the return value of model.
predictions = Predictive(model, guide=guide, num_samples=10)()

In [12]:
predictions

{'a_beta': tensor([1.0735, 1.0938, 1.0870, 1.0595, 1.0415, 1.0957, 1.0503, 1.0687, 1.0486,
         1.0633]),
 'a_gamma': tensor([0.9743, 0.9891, 0.9675, 0.9993, 0.9428, 0.9425, 0.9149, 0.9722, 1.0373,
         0.9123]),
 'I_sol': tensor([[0.0109, 0.0117, 0.0122, 0.0046, 0.6387, 0.2889, 0.1095, 0.0414, 0.0156],
         [0.0109, 0.0118, 0.0124, 0.0046, 0.6345, 0.2842, 0.1061, 0.0395, 0.0147],
         [0.0111, 0.0121, 0.0129, 0.0049, 0.6396, 0.2889, 0.1102, 0.0419, 0.0159],
         [0.0105, 0.0108, 0.0109, 0.0040, 0.6332, 0.2857, 0.1057, 0.0390, 0.0144],
         [0.0109, 0.0117, 0.0123, 0.0048, 0.6473, 0.2983, 0.1166, 0.0455, 0.0177],
         [0.0115, 0.0130, 0.0143, 0.0056, 0.6441, 0.2929, 0.1145, 0.0447, 0.0174],
         [0.0113, 0.0125, 0.0136, 0.0055, 0.6527, 0.3033, 0.1218, 0.0488, 0.0196],
         [0.0108, 0.0116, 0.0121, 0.0046, 0.6394, 0.2898, 0.1100, 0.0417, 0.0158],
         [0.0100, 0.0098, 0.0095, 0.0034, 0.6228, 0.2804, 0.0999, 0.0355, 0.0126],
         [0.0114, 0.012

In [13]:
model()

{'I': tensor([0.0090, 0.0081, 0.0072, 0.0026, 0.6279, 0.2985, 0.1096, 0.0401, 0.0146]),
 'R': tensor([0.0096, 0.0182, 0.0259, 0.0305, 0.2116, 0.7003, 0.8903, 0.9598, 0.9853]),
 'S': tensor([9.8138e-01, 9.7368e-01, 9.6691e-01, 9.6691e-01, 1.6056e-01, 1.2547e-03,
         1.9023e-04, 9.5335e-05, 7.4055e-05])}

In [ ]:
# SCAFFOLDING FOR DYNAMIC EVENT HANDLING BELOW

import pyro
from torch import Tensor
from torchdiffeq import odeint_event, odeint

Time = Tensor
State = tuple[Tensor, ...]

class BaseODEModel(pyro.nn.PyroModule):

    def __init__(self, static_events: list[Event]):
        super().__init__()
        # This is a list of events that are always sorted by time.
        # TODO: probably pre-sort this list just in case.
        self.static_events = static_events

    def deriv(self, t: Time, state: State) -> State:
        raise NotImplementedError

    @pyro.nn.pyro_method
    def param_prior(self):
        raise NotImplementedError

    @pyro.nn.pyro_method
    def initial_conditions_prior(self):
        raise NotImplementedError

    # @pyro.nn.pyro_method
    # def observation_model(self, state: State, tspan: Time, ?) -> ?: …

    def solve(self, initial_state: State, initial_time: Time) -> tuple[State, State]:
        
        current_state = initial_state
        current_time = initial_time

        solution = torch.tensor([])

        for i, static_event in enumerate(self.static_events):
            
            # TODO: change below
            # Note: Immediate goal is to get self.solve() to generate ode solutions that stop at the event times.
            # Note: Next we will add log likelihood at those events using an observation model.

            # Note: each static_event is an Event object, which has a forward() method that returns a Tensor.
            # TODO: Chad, could you please make this actually make sense.
            event_time, event_solution = odeint(self.deriv, current_state, current_time, event_fn=static_event)
    
            # TODO: Add log likelihood for event_solution.
            # Note: This will be done by adding an observation model to the ODE model and calling it with the event_solution as an argument.

            solution = torch.cat([solution, event_solution], dim=-1)
            # current = event_solution[-1]


    def forward(self, initial_state: State, initial_time: Time) -> tuple[State, State]:
        # Sample parameters from the prior. These parameters are generated as attributes of the model.
        self.param_prior()

        # TODO: Sample initial conditions from the prior instead of taking them as input.

        # Solve the ODE, taking into account any intervention and conditioning events.
        return self.solve(initial_state, initial_time)




In [ ]:
event_time, event_solution = odeint(self.deriv, current_state, current_time, event_fn=static_event)

In [ ]:

# TODO: Chad, initialize a stop event here.
# stop_event = ...

# TODO: Chad, create a model the subclasses BaseODEModel and has all of the above missing methods implemented.
# deriv, param_prior, initial_conditions_prior, observation_model
# class ActualODEModel(BaseODEModel):
#   ...

# TODO: Chad, instantiate the model here.
# model = ActualODEModel([stop_event])

# TODO: Chad, Call model.forward() here.
# solution, _ = model(initial_state, tspan)

In [7]:
observations = [ObservationEvent(torch.tensor([2.1]), {"I": torch.tensor([0.1])})]


In [8]:
conditioned_model = pyciemss.condition(model, observations)

AttributeError: module 'pyciemss' has no attribute 'condition'